In [9]:
import json
import jsonlines
from collections import Counter
from functools import reduce
import numpy as np
import re
from os import listdir
from os import close
import codecs 
import mediawiki
from joblib import Parallel, delayed
import sys

wikipedia = mediawiki.MediaWiki(lang=u'ru')

In [32]:
import urllib.request
from urllib.parse import quote

def request_categories(ids):
    url = "https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids="
    url = url + '|'.join(ids)
    print(url)
    p = urllib.request.urlopen(url).read()
    pages = json.loads(p)['query']['pages']
    result = {}
    for id in ids:
        try:
            article = pages[id]['revisions'][-1]['*']
            categories = re.findall(r'\[\[Категория:(.*)\]\]', article)
            if len(categories) > 0:
                result[id] = categories
                
        except KeyboardInterrupt:
            sys.exit(0)
            
        except:
            print(id + " not present")
    
    return result

In [35]:
request_categories(["71", "76"])

https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=71|76


{'71': ['Москва| ',
  'Населённые пункты, основанные в 1147 году',
  'Города, основанные в XII веке',
  'Города Древней Руси',
  'Города, в которых проходили летние Олимпийские игры 1980',
  'Столицы летних Универсиад',
  'Города-миллионеры России',
  'Областные центры России',
  'Города федерального значения России',
  'Золотое кольцо России',
  'Города из Списка русских городов дальних и ближних']}

In [ ]:
path = 'wiki_extracted1/'
alf = listdir(path)


def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]


def walker(a):
    categories = []
    page_ids = []
    articles = []
    _path = path + a
    __path = [_path + '/' + p for p in listdir(_path)]
    print(a, end=' ')
    for p in __path:
        print(p[18:], end=' ')
        with jsonlines.open(p, 'r') as f:
            ids = []
            titles = []
            texts = []
            for file in f:
                ids.append(file['id'])
                titles.append(file['title'])
                texts.append(file['text'])
            
            result = {}
            for chunk in chunks(ids, 4):
                result = request_categories(chunk)
            print(result)
            '''
            file = re.sub(r'\n+', '. ', file)
            file = re.split(r'</doc>', file)
            for f in file[:-1]:
                id = re.findall(r'id="(.*)" url', f)[0]
                try:
                    categories.extend(wikipedia.page(pageid=int(id)).categories)
                    page_ids.append(int(id))
                    articles.append(re.findall(r'">\. (.*)', f)[0])
                except KeyboardInterrupt:
                    close()
                except:
                    print("Page with id " + str(id) + " not found")
            '''
        break
    
    #return categories, page_ids, articles

for a in alf:
    walker(a)
#categories, page_ids, articles = Parallel(n_jobs=4)(delayed(walker)(a) for a in alf)

BP /wiki_44 https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=5316971|5316972|5316976|5316977
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=5316981|5316989|5316991|5316992
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=5316997|5316998|5317003|5317006
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=5317020|5317024|5317032|5317036
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=5317037|5317038|5317040|5317042
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=5317045|5317047|5317048|5317053
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=5317054|5317055|5317057|5317059
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&

https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=5318120|5318128|5318130|5318138
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=5318139|5318140|5318141|5318143
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=5318146|5318148|5318149|5318150
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=5318154|5318155|5318157|5318159
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=5318161|5318164|5318165|5318166
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=5318181|5318183|5318195|5318205
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=5318208|5318220|5318227|5318230
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&

https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=3588551|3588554|3588560|3588576
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=3588579|3588580|3588581|3588584
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=3588592|3588596|3588604|3588608
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=3588609|3588615|3588624|3588626
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=3588628|3588632|3588637|3588638
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=3588641|3588644|3588649|3588650
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=3588651|3588664|3588674|3588676
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&

https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=4242030|4242035|4242043|4242050
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=4242051|4242055|4242067|4242069
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=4242078|4242090|4242093|4242096
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=4242097|4242099|4242113|4242116
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=4242120|4242121|4242123|4242128
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=4242129|4242130|4242133|4242134
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=4242141|4242147|4242151|4242159
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&

https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=91384|91386|91387|91388
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=91390|91391|91393|91394
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=91395|91396|91400|91401
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=91404|91408|91412|91415
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=91416|91422|91426|91432
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=91433|91443|91445|91451
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=91455|91457|91466|91470
https://ru.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&pageids=91472|91473|91476|91481
https://ru.wikipedia.org

In [19]:
#with open('output.json') as f:
    #info = json.load(f)
    #print(len(info))
    #categories = [i["categories"] for i in info]
    #categories = reduce(lambda x,y:x+y, categories)
categories = [c[10:] for c in categories]
print(sorted(set([c for c in categories if re.findall(r'.* /d/d/d/d .*', c)])))
counts = Counter(categories)
print(np.mean(list(counts.values())))
print(counts)

NameError: name 'categories' is not defined